# Exploring the Queensland naturalisation index from Queensland State Archives

In [this notebook](qld_add_series_info_to_naturalisations_index.ipynb) we loaded the Naturalisation Index from the Queensland government data portal and added a series identifier to each record.

Now that we've associated each entry in the original dataset with a series, we can break the data down by series to better understand the content of the index.

In [2]:
import pandas as pd
import altair as alt

Load the dataset we created with series identifiers.

In [3]:
df = pd.read_csv('qsa_naturalisations_index_with_series.csv')

Show the number of entries per series.

In [46]:
pd.set_option('display.max_colwidth', None)
series_counts = df.value_counts(['series_id', 'series_title']).to_frame().reset_index()
series_counts.columns = ['series_id', 'series_title', 'count']
series_counts

,series_id,series_title,count
0,5177,Registers of Aliens to Whom Oaths of Allegiance for Naturalisation Were Administered,14325
1,5741,Oaths of Allegiance Sworn by Aliens Being Naturalised,10344
2,7224,Oaths of Allegiance,734
3,8400,Special Batches,528
4,9403,Naturalisation Files,258
5,5743,Certificates of Naturalisation and Associated Papers,230
6,5745,"Applications for Copies of Records of Naturalisation, and Related Correspondence",131
7,7164,Register of Fees of Office,122
8,5253,Inwards Correspondence,82
9,12748,"Letters Addressed to the Government Resident by the Colonial Secretary, Sydney",15


In [44]:
alt.Chart(series_counts).mark_bar().encode(
    x='series_id:N',
    y='count:Q',
    color='series_id:N',
    tooltip=['series_id', 'series_title', 'count']
)

alt.Chart(...)

## Visualise records over time

In [12]:
year_counts = df.value_counts(['Year', 'series_id']).to_frame().reset_index()
year_counts.columns = ['year', 'series', 'count']
year_counts

,year,series,count
0,1886,5741,648
1,1886,5177,635
2,1883,5177,572
3,1888,5741,569
4,1888,5177,564
...,...,...,...
220,1874,5743,1
221,1873,5745,1
222,1873,5743,1
223,1872,5745,1


See that '18889' at the end? Looks like there's some dodgy dates. Let's sort by year to see what the limits should be.

In [13]:
year_counts.sort_values('year')

,year,series,count
204,0,5745,1
153,0,5741,5
169,0,5743,4
166,1851,12748,4
178,1853,12748,3
...,...,...,...
203,1904,7164,1
96,1904,8400,38
170,1905,8400,4
205,1969,5741,1


So, it looks like we should filter out records before 1851 and after 1905.

Let's apply that filter and visualise the results for all series in a stacked bar chart.

In [65]:
alt.Chart(year_counts.loc[(year_counts['year'] >= 1851) & (year_counts['year'] <= 1905)]).mark_bar(size=10).encode(
    x=alt.X('year:Q', axis=alt.Axis(format='c')),
    y='count:Q',
    color='series:N',
    tooltip=['year', 'series', 'count']
).properties(width=700)

alt.Chart(...)

To make the different date ranges clearer, we can facet the results by series.

In [47]:
alt.Chart(year_counts.loc[(year_counts['year'] >= 1851) & (year_counts['year'] <= 1905)]).mark_bar(size=5).encode(
    x=alt.X('year:Q', axis=alt.Axis(format='c')),
    y='count:Q',
    color='series:N',
    facet=alt.Facet('series:N', columns=2),
    tooltip=['year', 'count']
).properties(width=350, height=150)

alt.Chart(...)

We can see that series 5177 and 5745, which both record Oaths of Allegiance, have similar distributions, though there are significant differences. On way we might be able to create a more complete time series would be to combine the results of these two series, then remove duplicates based on name and year.

First we filter the dataset to include just these series.

In [55]:
df_oaths = df.copy().loc[df['series_id'].isin([5177, 5741])]
# How many records?
df_oaths.shape[0]

24669

Then we remove duplicates based on name and year.

In [56]:
df_oaths.drop_duplicates(subset=['Last name', 'Given names', 'Year'], inplace=True)
# How many now?
df_oaths.shape[0]

17755

We can now visualise the deduplicated dataset.

In [63]:
oath_year_counts = df_oaths.value_counts(['Year', 'series_id']).to_frame().reset_index()
oath_year_counts.columns = ['year', 'series', 'count']

In [66]:
alt.Chart(oath_year_counts.loc[(oath_year_counts['year'] >= 1851) & (oath_year_counts['year'] <= 1905)]).mark_bar(size=10).encode(
    x=alt.X('year:Q', axis=alt.Axis(format='c')),
    y='count:Q',
    tooltip=['year', 'count']
).properties(width=700)

alt.Chart(...)